In [2]:
import tweepy
import time
import sqlite3
from contextlib import closing
import time
from tweepy import TweepError
from dotenv import load_dotenv
import os
load_dotenv()

In [3]:
consumer_key= os.getenv('consumer_key')
consumer_secret = os.getenv('consumer_secret')
access_token = os.getenv('access_token')
access_token_secret = os.getenv('access_token_secret')

In [4]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [5]:
id_lst =[]
single_call_count = 0
single_call_limit = 100
total_limit_count = 0
total_limit_requests = 899
count = 1

In [6]:

def packaging_data_for_update(hydrated_tweets):
    list_to_update = []
    good_set = set()
    for tweet in hydrated_tweets:
        try:
            text = tweet.retweeted_status.full_text
        except AttributeError:
            text = tweet.full_text
        Created_at = tweet.created_at
        location = tweet.user.location
        followers_count = int(tweet.user.followers_count)
        tweet_id = int(tweet.id)
        good_set.add(int(tweet_id))
        User_id = int(tweet.user.id)
        try:
            RT = tweet.retweeted_status.id
        except AttributeError:
            RT = 'False'
        try:
            retweet_count = tweet.retweeted_status.retweet_count
        except AttributeError:
            retweet_count = tweet.retweet_count
        list_to_update.append([Created_at, location, followers_count, RT, User_id, text, retweet_count, tweet_id])
    return list_to_update, good_set

In [7]:
with closing(sqlite3.connect('BigDataChallenge.sqlite')) as cnxn:
    with closing(cnxn.cursor()) as cursorIter:
        # cursorIter.execute('Select Tweet_ID from Sentiment where User_id is not null ')
        cursorIter.execute('Select Tweet_ID from Sentiment where Fetched is null ')
        for row in cursorIter:
            if single_call_count >= single_call_limit:
                print(f'Fetching : {count} calls made')
                try:
                    hydrated_tweets = api.statuses_lookup(id_lst, tweet_mode = "extended")
                except TweepError:
                    time.sleep(60)
                    hydrated_tweets = api.statuses_lookup(id_lst, tweet_mode = "extended")
                packaged_info, good_set = packaging_data_for_update(hydrated_tweets)
                with closing(cnxn.cursor()) as cursorUpdater:
                    cursorUpdater.executemany('update Sentiment set Created_at = ?, location = ?, followers_count = ?, Original_tweet = ?, User_id = ?, Content = ?, Retweet_count = ?, Fetched = "Yes" where Tweet_ID = ?', packaged_info)
                    cursorUpdater.executemany('update Sentiment set Fetched = "Yes" where Tweet_ID = ?', [[tweet] for tweet in set(id_lst) - good_set])
                single_call_count = 1
                id_lst = [row[0]]
                count += 1
                cnxn.commit()
            else:
                single_call_count += 1
                id_lst.append(row[0])


lls made
Fetching : 6445 calls made
Fetching : 6446 calls made
Fetching : 6447 calls made
Fetching : 6448 calls made
Fetching : 6449 calls made
Fetching : 6450 calls made
Fetching : 6451 calls made
Fetching : 6452 calls made
Fetching : 6453 calls made
Fetching : 6454 calls made
Fetching : 6455 calls made
Fetching : 6456 calls made
Fetching : 6457 calls made
Fetching : 6458 calls made
Fetching : 6459 calls made
Fetching : 6460 calls made
Fetching : 6461 calls made
Fetching : 6462 calls made
Fetching : 6463 calls made
Fetching : 6464 calls made
Fetching : 6465 calls made
Fetching : 6466 calls made
Fetching : 6467 calls made
Fetching : 6468 calls made
Fetching : 6469 calls made
Fetching : 6470 calls made
Fetching : 6471 calls made
Fetching : 6472 calls made
Fetching : 6473 calls made
Fetching : 6474 calls made
Fetching : 6475 calls made
Fetching : 6476 calls made
Fetching : 6477 calls made
Fetching : 6478 calls made
Fetching : 6479 calls made
Fetching : 6480 calls made
Fetching : 6481 cal

OperationalError: database is locked

In [7]:
lst1 = set([1, 2 ,3])
lst2 = set([2, 3, 4])
print([x for x in lst2])

[2, 3, 4]


In [13]:
example = api.statuses_lookup([1289350148263251969, 1289350148334657537], tweet_mode="extended")


In [54]:
# print(example[0].retweeted_status.full_text)
try:
    print(example[1].retweeted_status.id)
except AttributeError:
    print(example[0].retweet_count)
# followers_count = int(tweet._json['user']['followers_count'])
# User_id = int(tweet._json['user']['id'])
# RT = bool(re.match('^RT', tweet._json['text']))
# tweet_id = int(tweet._json['id'])
# text = tweet._json['text']
print(example[0].retweeted_status)

1289309756415336448


AttributeError: 'Status' object has no attribute 'retweeted_status'

In [38]:
import json
with open('test.txt', 'w') as f:
    json.dump(example[0]._json, f)
    json.dump(example[1]._json, f)

In [10]:
import json
import re
status = example[0]
print(status._json['created_at'])
print(status._json['user']['location'])
print(status._json['user']['followers_count'])
print(status._json['user']['id'])
print(re.match('^RT', status._json['text']))
print(status._json['id'])
# text
# created_at
# location
# followers_count

IndexError: list index out of range